In [5]:
import subprocess
import io
import fetch_data as fd
import numpy as np


categories = ['earn','crude']
numberOfTraining = [10,10]
numberOfTesting = [7,2]


trainData,trainLabels, testData,testLabel = fd.loadData(categories,numberOfTraining,numberOfTesting)
# from subprocess import Popen, PIPE, STDOUT

# Compile the fast_recursive_kernel
test = subprocess.Popen(["g++","fast_recursive_kernel.cpp","-o","fast_recursive_kernel.out"], stdout=subprocess.PIPE)
output_compile = test.communicate()[0]
# print(output_compile.decode('ascii'))


def parseMatrix(matrix):
    num_rows = int(matrix[0])
    num_cols = int(matrix[1])
    K = np.zeros([num_rows,num_cols])
    counter = 2
    for i in range(num_rows):
        for j in range(num_cols):
            K[i,j] = float(matrix[counter])
            counter = counter + 1
    return K

In [6]:
# Test fast_recursive_kernel
n = 2
l = 0.5

def marcus_recursive_kernel(s,t,n,l):
    the_strings = b''
    the_args = bytes(str(len(s)),'ascii')+ b' ' + bytes(str(len(t)),'ascii')+ b' '
    for i in s:
        the_strings = the_strings + bytes(i,'ascii')
        the_args = the_args + bytes(str(len(i)),'ascii') + b' '
    for i in t:
        the_strings = the_strings + bytes(i,'ascii')
        the_args = the_args + bytes(str(len(i)),'ascii') + b' '
    the_args = the_args + bytes(str(n), 'ascii')+ b' ' + bytes(str(l), 'ascii')
    fast_kernel = subprocess.Popen(["./fast_recursive_kernel.out"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    output_test = fast_kernel.communicate(input=b'1 '+the_args+the_strings)[0]
    output_test_list = output_test.decode('utf-8').split()
    K = parseMatrix(output_test_list)
    return K

K = marcus_recursive_kernel(trainData,trainData,n,l)


In [10]:
%timeit marcus_recursive_kernel(trainData,trainData,5,l)

7.6 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
print(K)

[[ 1.          0.6298942   0.59103979  0.76254216  0.60082857  0.52362418
   0.76392863  0.67396455  0.47040547  0.81713632  0.45916014  0.75342122
   0.81265395  0.5445702   0.70009494  0.71414453  0.27579864  0.73925103
   0.72214739  0.77579749]
 [ 0.6298942   1.          0.6452375   0.63146551  0.53402018  0.7877384
   0.50737407  0.81291982  0.88093318  0.66147496  0.47261162  0.61243066
   0.60610604  0.55728187  0.56797623  0.60817103  0.22297053  0.58745533
   0.58674142  0.58212458]
 [ 0.59103979  0.6452375   1.          0.64315049  0.55412792  0.56031133
   0.49586927  0.6180585   0.60154527  0.61651005  0.42166653  0.54856985
   0.5859281   0.52252272  0.51938264  0.5863897   0.23316702  0.57656934
   0.51430811  0.56931677]
 [ 0.76254216  0.63146551  0.64315049  1.          0.57968236  0.5470677
   0.66785327  0.68611083  0.54857958  0.78173585  0.47287187  0.74826831
   0.78503169  0.6258081   0.67833404  0.70358177  0.32178909  0.75015512
   0.71757202  0.74671689]
 [ 0.6